# Adapting NanoGPT

In [ ]:
import importlib
import sys

## Intro
The goal of this notebook is to show how to use an arbitrary model with the Oumi training loop.

In this case, we will adapt [nanogpt](https://github.com/karpathy/nanoGPT), and train it using both the Oumi and HuggingFace training loops.

## Setup

This notebook assumes that you have already installed the `oumi` package. If you haven't, you can install it by running `!pip install oumi`.

We start then by cloning the nanoGPT repository, and adding nanoGPT to our python path


In [ ]:
from pathlib import Path

module_folder = Path("/tmp/oumi/nanoGPT")

# Clone the nanoGPT repo
if not module_folder.is_dir():
    module_folder.mkdir(parents=True, exist_ok=True)
    !git clone https://github.com/karpathy/nanoGPT {module_folder}
else:
    print("nanoGPT already cloned!")

sys.path.append(str(module_folder))

Next we install the required dependencies. 

In [ ]:
if importlib.util.find_spec("tiktoken") is not None:
    print("tiktoken is already installed!")
else:
    %pip install tiktoken

## Adapting nanoGPT model

In [ ]:
import torch.nn.functional as F
from model import GPT, GPTConfig  # import from ~/nanoGPT/model.py

from oumi.core import registry


@registry.register("oumi-nanoGPT", registry_type=registry.RegistryType.MODEL)
class OumiNanoGPT(GPT):
    def __init__(self, **kwargs):
        """Initializes an instance of the class."""
        gpt_config = GPTConfig()
        gpt_config.bias = False

        super().__init__(gpt_config)

    def forward(self, input_ids, labels=None, attention_mask=None):
        """Performs the forward pass of the model."""
        # Update the return format to be compatible with our Trainer.
        logits, loss = super().forward(idx=input_ids, targets=labels)
        outputs = {"logits": logits}
        if loss:
            outputs["loss"] = loss
        return outputs

    def criterion(self):
        """Returns the criterion used for calculating the loss."""
        return F.cross_entropy

## Training

Ok now we are ready to train our model! we can start from the default gpt2 config, and edit as needed.

In [ ]:
import oumi
from oumi.core.configs import TrainerType, TrainingConfig

In [ ]:
# Starting from the default GPT-2 config
config_path = "../configs/recipes/gpt2/pretraining/mac_train.yaml"
config = TrainingConfig.from_yaml(config_path)

# Update to use our newly registered nanoGPT model
config.model.model_name = "oumi-nanoGPT"  # needs to match the registered model name

# We do not have a custom tokenizer, but we can use the GPT-2 tokenizer from HuggingFace
config.model.tokenizer_name = "gpt2"

config.training.trainer_type = TrainerType.OUMI
config.training.max_steps = 10
config.training.logging_steps = 1
config.training.gradient_accumulation_steps = 1
config.training.enable_wandb = False
config.training.enable_tensorboard = False

In [ ]:
oumi.train(config)